# Imports

In [1]:
import os

import pandas as pd
import scanpy as sc
sc.logging.print_versions()
sc.logging.print_header()

outdir_gdrive = '/home/olga/googledrive/TabulaMicrocebus/data/cross-species'
outdir_local = '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species'
outdirs = outdir_gdrive, outdir_local

-----
anndata     0.7.5
scanpy      1.7.0rc1
sinfo       0.3.1
-----
anndata             0.7.5
backcall            0.1.0
cairo               1.18.2
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.0
decorator           4.4.1
get_version         2.1
h5py                2.10.0
igraph              0.7.1
importlib_metadata  0.19
ipykernel           5.1.3
ipython_genutils    0.2.0
jedi                0.15.1
joblib              0.14.1
kiwisolver          1.1.0
legacy_api_wrap     0.0.0
leidenalg           0.7.0
llvmlite            0.30.0
louvain             0.6.1
matplotlib          3.1.2
more_itertools      NA
mpl_toolkits        NA
natsort             6.2.0
numba               0.46.0
numexpr             2.7.0
numpy               1.17.3
packaging           20.8
pandas              1.1.4
parso               0.5.1
pexpect             4.7.0
pickleshare         0.7.5
pkg_resources       NA
prompt_toolkit      3.0.2
ptyprocess          0.6.0
pyg

## Instructions from Camille

1. Cross_species_unified_annotations_Lung_Muscle_Blood.xlsx where you can see the updated unified annotations (I tried my best to look for any typos!!)
1. tabula-muris-senis_10X_Limb_Muscle_updated-annotations.h5ad
    1. Olga please remove 1 and 3 month old data
    1. ***Note there are only 5 slow muscle cells in the TMS data, so depending on your cutoff Olga for minimum number of cells, we might not be able to compare slow vs fast muscle cells. 
1. tabula-sapiens_P1P2_10X_SS2_Muscle_updated-annotations.h5ad
    1. This is only p1 and p2. Olga please remove diaphragm data from P2 and all the SmartSeq2 data.
    1. Antoine under 'Anatomical information' for P2 there is: rectus, diaphragm and 'nan'. Not sure what 'nan' refers to in that case. But it's all smartseq2 so it will removed. 

In [2]:
ls -lha /mnt/ibm_lg/angela/sapiens/data/uploads_to_s3/v2_july_2021

total 119G
drwxr-xr-x 2 angela 21040 4.0K Jul 12 10:34  ./
drwxr-xr-x 4 angela 21040 4.0K Jul 11 22:50  ../
-rw-r--r-- 1 angela 21040 1.8G Jul 12 10:29  TS_Bladder.h5ad
-rw-r--r-- 1 angela 21040 2.9G Jul 12 10:32  TS_Blood.h5ad
-rw-r--r-- 1 angela 21040 974M Jul 12 10:34  TS_Bone_Marrow.h5ad
-rw-r--r-- 1 angela 21040 1.2G Jul 12 10:23  TS_Eye.h5ad
-rw-r--r-- 1 angela 21040 2.1G Jul 12 10:24  TS_Fat.h5ad
-rw-r--r-- 1 angela 21040 1.4G Jul 12 10:24  TS_Heart.h5ad
-rw-r--r-- 1 angela 21040 847M Jul 12 10:33  TS_Kidney.h5ad
-rw-r--r-- 1 angela 21040 873M Jul 12 10:23  TS_Large_Intestine.h5ad
-rw-r--r-- 1 angela 21040 405M Jul 12 10:28  TS_Liver.h5ad
-rw-r--r-- 1 angela 21040 3.7G Jul 12 10:34  TS_Lung.h5ad
-rw-r--r-- 1 angela 21040 3.4G Jul 12 10:30  TS_Lymph_Node.h5ad
-rw-r--r-- 1 angela 21040 938M Jul 12 10:28  TS_Mammary.h5ad
-rw-r--r-- 1 angela 21040 2.4G Jul 12 10:27  TS_Muscle.h5ad
-rw-r--r-- 1 angela 21040 1.4G Jul 12 10:25  TS_Pancreas.h5ad
-rw-r--r-- 1 angela 21040 1.4G Jul 12 10:

## Read blood data

In [3]:
h5ad = os.path.join(
    "/mnt/ibm_lg/angela/sapiens/data/uploads_to_s3/v2_july_2021",
    "TS_Blood.h5ad",
)

blood = sc.read(h5ad, cache=True)
blood

AnnData object with n_obs × n_vars = 50115 × 58870
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'dendrogram_consensus_prediction', 'dendrogram_tissue_cell_type', 'donor_colors', 'donor_method_colors', 'hvg', 'method_colors', 'neighbors', 'sex_colors', 'tissue_colors', 'umap'
    obsm: 'X_pca', 'X_scvi', 'X_scvi_umap', 'X_umap'
    layers: 'decontXcounts', 'raw_counts'
    obsp: 'connectivities', 'distances'

## Read Camille's annotations

In [4]:
h5ad = os.path.join(
    "/home/olga/googledrive/TabulaMicrocebus/data/cross-species/unified_annotations/",
    "tabula-sapiens_P1P2_10X_SS2_Muscle_updated-annotations.h5ad",
)

sapiens = sc.read(h5ad, cache=True)
sapiens

AnnData object with n_obs × n_vars = 20197 × 58833
    obs: 'Annotation', 'Predictability', 'Manually Annotated', 'Donor', 'Method', 'Organ', 'Compartment', 'Anatomical Information', 'TS_muscle_CE_compartment', 'TS_muscle_CE_anatomical_information', 'TS_muscle_CE_free_annotation'
    var: 'gene_symbol', 'ensembl_id', 'gene_length'
    uns: 'Annotation_colors', 'Compartment_colors', 'Donor_colors', 'Manually Annotated_colors', 'Method_colors', 'Organ_colors', 'Propagated.Annotationcollapsed_colors', 'TS_muscle_CE_compartment_colors', 'TS_muscle_CE_free_annotation_colors', '_scvi', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'tissue_colors', 'umap'
    obsm: 'X_umap'
    layers: 'counts', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [5]:
sapiens.layers['raw_counts']

<20197x58833 sparse matrix of type '<class 'numpy.float32'>'
	with 49789479 stored elements in Compressed Sparse Row format>

In [6]:
sapiens.to_df(layer='raw_counts')

,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,MIR1302-2,FAM138A,OR4G4P,OR4G11P,OR4F5,AL627309.1,...,AMD1P2-1,DPH3P2-1,VAMP7-1,ELOCP24-1,TRPC6P-1,IL9R-1,AJ271736.1-1,WASIR1-1,WASH6P-1,DDX11L16-1
CTTCTAAAGTGACACG_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TSP2_Muscle_rectusabdominus_SS2_B113693_B111573_Stromal_E18_S114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACGCACGTCTAGAGCT_TSP1_muscle_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GGCAGTCAGTCGTTAC_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TSP2_Muscle_rectusabdominus_SS2_B113693_B111573_Stromal_K5_S245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTCACGGGTGACATCT_TSP2_Muscle_rectusabdominus_10X_1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCCACACATAACC_TSP2_Muscle_rectusabdominus_10X_1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTCATGACAACCGATT_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CATCAAGTCTGTGCTC_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Subset to 10x

In [7]:
sapiens.obs.Method.unique()

['10X', 'smartseq2']
Categories (2, object): ['10X', 'smartseq2']

In [8]:
sapiens_10x = sapiens[sapiens.obs.Method == "10X"]
sapiens_10x

View of AnnData object with n_obs × n_vars = 17379 × 58833
    obs: 'Annotation', 'Predictability', 'Manually Annotated', 'Donor', 'Method', 'Organ', 'Compartment', 'Anatomical Information', 'TS_muscle_CE_compartment', 'TS_muscle_CE_anatomical_information', 'TS_muscle_CE_free_annotation'
    var: 'gene_symbol', 'ensembl_id', 'gene_length'
    uns: 'Annotation_colors', 'Compartment_colors', 'Donor_colors', 'Manually Annotated_colors', 'Method_colors', 'Organ_colors', 'Propagated.Annotationcollapsed_colors', 'TS_muscle_CE_compartment_colors', 'TS_muscle_CE_free_annotation_colors', '_scvi', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'tissue_colors', 'umap'
    obsm: 'X_umap'
    layers: 'counts', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [9]:
sapiens_10x.obs.TS_muscle_CE_anatomical_information.value_counts()

rectusabdominus    13215
diaphragm           4164
Name: TS_muscle_CE_anatomical_information, dtype: int64

In [10]:
sapiens_10x[:10, ['TTN', 'MYL3', 'MYL6']].to_df(layer='raw_counts')

,TTN,MYL3,MYL6
CTTCTAAAGTGACACG_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,13.0
ACGCACGTCTAGAGCT_TSP1_muscle_3,2.0,0.0,1.0
GGCAGTCAGTCGTTAC_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,11.0
TCAGTGAGTCGGAAAC_TSP1_muscle_2,0.0,0.0,0.0
TCAGTCCAGTTGTACC_TSP2_Muscle_diaphragm_10X_1_2,0.0,0.0,7.0
AACCAACAGACTCTAC_TSP1_muscle_1,0.0,0.0,4.0
CCTCCTCCATTCCTCG_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,4.0
CGATGCGTCTTCTGTA_TSP2_Muscle_diaphragm_10X_1_1,0.0,0.0,1.0
TATCCTACAGCTCGGT_TSP2_Muscle_rectusabdominus_10X_1_2,1.0,0.0,6.0
CTGTAGACAGCCGTTG_TSP2_Muscle_diaphragm_10X_1_2,0.0,0.0,6.0


## Assign raw counts to `X`

In [11]:
sapiens_10x.X = sapiens_10x.layers['raw_counts']
del sapiens_10x.layers['counts']

In [12]:
sapiens_10x

AnnData object with n_obs × n_vars = 17379 × 58833
    obs: 'Annotation', 'Predictability', 'Manually Annotated', 'Donor', 'Method', 'Organ', 'Compartment', 'Anatomical Information', 'TS_muscle_CE_compartment', 'TS_muscle_CE_anatomical_information', 'TS_muscle_CE_free_annotation'
    var: 'gene_symbol', 'ensembl_id', 'gene_length'
    uns: 'Annotation_colors', 'Compartment_colors', 'Donor_colors', 'Manually Annotated_colors', 'Method_colors', 'Organ_colors', 'Propagated.Annotationcollapsed_colors', 'TS_muscle_CE_compartment_colors', 'TS_muscle_CE_free_annotation_colors', '_scvi', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'tissue_colors', 'umap'
    obsm: 'X_umap'
    layers: 'raw_counts'
    obsp: 'connectivities', 'distances'

### Double check that data is raw

In [13]:
sapiens_10x[:10, ['TTN', 'MYL3', 'MYL6']].to_df()

,TTN,MYL3,MYL6
CTTCTAAAGTGACACG_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,13.0
ACGCACGTCTAGAGCT_TSP1_muscle_3,2.0,0.0,1.0
GGCAGTCAGTCGTTAC_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,11.0
TCAGTGAGTCGGAAAC_TSP1_muscle_2,0.0,0.0,0.0
TCAGTCCAGTTGTACC_TSP2_Muscle_diaphragm_10X_1_2,0.0,0.0,7.0
AACCAACAGACTCTAC_TSP1_muscle_1,0.0,0.0,4.0
CCTCCTCCATTCCTCG_TSP2_Muscle_rectusabdominus_10X_1_1,0.0,0.0,4.0
CGATGCGTCTTCTGTA_TSP2_Muscle_diaphragm_10X_1_1,0.0,0.0,1.0
TATCCTACAGCTCGGT_TSP2_Muscle_rectusabdominus_10X_1_2,1.0,0.0,6.0
CTGTAGACAGCCGTTG_TSP2_Muscle_diaphragm_10X_1_2,0.0,0.0,6.0


## Remove diaphragm from pilot 2

In [14]:
diaphgram_pilot2 = (sapiens_10x.obs.TS_muscle_CE_anatomical_information == "diaphragm") & (sapiens_10x.obs.Donor == "TSP2")
print(diaphgram_pilot2.sum())

sapiens_10x_no_diaphragm_pilot2 = sapiens_10x[~diaphgram_pilot2]
sapiens_10x_no_diaphragm_pilot2

4164


View of AnnData object with n_obs × n_vars = 13215 × 58833
    obs: 'Annotation', 'Predictability', 'Manually Annotated', 'Donor', 'Method', 'Organ', 'Compartment', 'Anatomical Information', 'TS_muscle_CE_compartment', 'TS_muscle_CE_anatomical_information', 'TS_muscle_CE_free_annotation'
    var: 'gene_symbol', 'ensembl_id', 'gene_length'
    uns: 'Annotation_colors', 'Compartment_colors', 'Donor_colors', 'Manually Annotated_colors', 'Method_colors', 'Organ_colors', 'Propagated.Annotationcollapsed_colors', 'TS_muscle_CE_compartment_colors', 'TS_muscle_CE_free_annotation_colors', '_scvi', 'donor_colors', 'leiden', 'method_colors', 'neighbors', 'tissue_colors', 'umap'
    obsm: 'X_umap'
    layers: 'raw_counts'
    obsp: 'connectivities', 'distances'

In [15]:
sapiens_10x_no_diaphragm_pilot2.obs.head()

,Annotation,Predictability,Manually Annotated,Donor,Method,Organ,Compartment,Anatomical Information,TS_muscle_CE_compartment,TS_muscle_CE_anatomical_information,TS_muscle_CE_free_annotation
CTTCTAAAGTGACACG_TSP2_Muscle_rectusabdominus_10X_1_1,capillary endothelial cell,1.0,True,TSP2,10X,Muscle,Endothelial,rectusabdominus,endothelial,rectusabdominus,capillary cell
ACGCACGTCTAGAGCT_TSP1_muscle_3,skeletal muscle satellite stem cell,1.0,True,TSP1,10X,Muscle,Stromal,nan,stromal,rectusabdominus,skeletal muscle satellite stem cell
GGCAGTCAGTCGTTAC_TSP2_Muscle_rectusabdominus_10X_1_1,mesenchymal stem cell,1.0,True,TSP2,10X,Muscle,Stromal,rectusabdominus,stromal,rectusabdominus,fibroadipogenic progenitor cell
TCAGTGAGTCGGAAAC_TSP1_muscle_2,mesenchymal stem cell,1.0,True,TSP1,10X,Muscle,Stromal,nan,stromal,rectusabdominus,fibroadipogenic progenitor cell
AACCAACAGACTCTAC_TSP1_muscle_1,pericyte cell,1.0,True,TSP1,10X,Muscle,Stromal,nan,stromal,rectusabdominus,pericyte


### Create unified "individual"

In [16]:
individual_mapper = {"TSP1": 'human_tsp_1', 'TSP2': "human_tsp_2"}

sapiens_10x_no_diaphragm_pilot2.obs['individual'] = sapiens_10x_no_diaphragm_pilot2.obs.Donor.map(individual_mapper)
sapiens_10x_no_diaphragm_pilot2.obs['individual'].value_counts()

Trying to set attribute `.obs` of view, copying.


human_tsp_2    10699
human_tsp_1     2516
Name: individual, dtype: int64

## Use minimal columns for `.obs` (cell metadata)

### Subset to minimal obs columns

In [17]:
obs_cols = [
    "individual",
    "Organ",
    'Method',
    "TS_muscle_CE_compartment",
    "TS_muscle_CE_anatomical_information",
    "TS_muscle_CE_free_annotation",
]

minimal_obs = sapiens_10x_no_diaphragm_pilot2.obs[obs_cols]
minimal_obs["age"] = "50-59y"
minimal_obs.head()

/home/olga/miniconda3/envs/tabula-microcebus-jan2021/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,individual,Organ,Method,TS_muscle_CE_compartment,TS_muscle_CE_anatomical_information,TS_muscle_CE_free_annotation,age
CTTCTAAAGTGACACG_TSP2_Muscle_rectusabdominus_10X_1_1,human_tsp_2,Muscle,10X,endothelial,rectusabdominus,capillary cell,50-59y
ACGCACGTCTAGAGCT_TSP1_muscle_3,human_tsp_1,Muscle,10X,stromal,rectusabdominus,skeletal muscle satellite stem cell,50-59y
GGCAGTCAGTCGTTAC_TSP2_Muscle_rectusabdominus_10X_1_1,human_tsp_2,Muscle,10X,stromal,rectusabdominus,fibroadipogenic progenitor cell,50-59y
TCAGTGAGTCGGAAAC_TSP1_muscle_2,human_tsp_1,Muscle,10X,stromal,rectusabdominus,fibroadipogenic progenitor cell,50-59y
AACCAACAGACTCTAC_TSP1_muscle_1,human_tsp_1,Muscle,10X,stromal,rectusabdominus,pericyte,50-59y


### Rename obs columns

In [18]:
minimal_obs = minimal_obs.rename(
    columns={
        "X10X_barcode": "cell_barcode",
        "X10X_run": "channel",
        'TS_muscle_CE_compartment': 'compartment',
        "TS_muscle_CE_anatomical_information": "subtissue",
        'TS_muscle_CE_free_annotation': 'free_annotation',
        "Method": "method",
        "Organ": "tissue",
    }
)
minimal_obs.head()

,individual,tissue,method,compartment,subtissue,free_annotation,age
CTTCTAAAGTGACACG_TSP2_Muscle_rectusabdominus_10X_1_1,human_tsp_2,Muscle,10X,endothelial,rectusabdominus,capillary cell,50-59y
ACGCACGTCTAGAGCT_TSP1_muscle_3,human_tsp_1,Muscle,10X,stromal,rectusabdominus,skeletal muscle satellite stem cell,50-59y
GGCAGTCAGTCGTTAC_TSP2_Muscle_rectusabdominus_10X_1_1,human_tsp_2,Muscle,10X,stromal,rectusabdominus,fibroadipogenic progenitor cell,50-59y
TCAGTGAGTCGGAAAC_TSP1_muscle_2,human_tsp_1,Muscle,10X,stromal,rectusabdominus,fibroadipogenic progenitor cell,50-59y
AACCAACAGACTCTAC_TSP1_muscle_1,human_tsp_1,Muscle,10X,stromal,rectusabdominus,pericyte,50-59y


### Add species

In [19]:
minimal_obs['species'] = "Human"
minimal_obs['species_latin'] = 'Homo sapiens'
minimal_obs.head()

,individual,tissue,method,compartment,subtissue,free_annotation,age,species,species_latin
CTTCTAAAGTGACACG_TSP2_Muscle_rectusabdominus_10X_1_1,human_tsp_2,Muscle,10X,endothelial,rectusabdominus,capillary cell,50-59y,Human,Homo sapiens
ACGCACGTCTAGAGCT_TSP1_muscle_3,human_tsp_1,Muscle,10X,stromal,rectusabdominus,skeletal muscle satellite stem cell,50-59y,Human,Homo sapiens
GGCAGTCAGTCGTTAC_TSP2_Muscle_rectusabdominus_10X_1_1,human_tsp_2,Muscle,10X,stromal,rectusabdominus,fibroadipogenic progenitor cell,50-59y,Human,Homo sapiens
TCAGTGAGTCGGAAAC_TSP1_muscle_2,human_tsp_1,Muscle,10X,stromal,rectusabdominus,fibroadipogenic progenitor cell,50-59y,Human,Homo sapiens
AACCAACAGACTCTAC_TSP1_muscle_1,human_tsp_1,Muscle,10X,stromal,rectusabdominus,pericyte,50-59y,Human,Homo sapiens


## Use minimum columns for `.var` (gene metadata)

In [20]:
sorted(sapiens_10x_no_diaphragm_pilot2.var.columns)

['ensembl_id', 'gene_length', 'gene_symbol']

In [21]:
var_cols = ['ensembl_id', 'gene_length', 'gene_symbol']

new_var = sapiens_10x_no_diaphragm_pilot2.var[var_cols]
new_var.head()

,ensembl_id,gene_length,gene_symbol
DDX11L1,ENSG00000223972.5,1144.5,DDX11L1
WASH7P,ENSG00000227232.5,1351.0,WASH7P
MIR6859-1,ENSG00000278267.1,68.0,MIR6859-1
MIR1302-2HG,ENSG00000243485.5,623.5,MIR1302-2HG
MIR1302-2,ENSG00000284332.1,138.0,MIR1302-2


# Add unified annotations

## Blood -- Don't have updated version for this -- Ignore

Celltypes in `/mnt/ibm_lg/angela/sapiens/data/uploads_to_s3/v2_july_2021` are different. No `cell_ontology_type` column

In [22]:
# import unified_annotations

# blood_grouping = unified_annotations.get_celltype_converter(
#     "Blood", ("Human", "Tabula Sapiens", "cell_ontology_type")
# ) 
# blood_grouping = blood_grouping.drop('tissue', axis=1)
# print(blood_grouping.shape)
# blood_grouping.head()

## Muscle

In [23]:
muscle_grouping = unified_annotations.get_celltype_converter(
    "Muscle",
    (
        "Human",
        "Tabula Sapiens",
        "TS_muscle_CE_free_annotation",
    ),
)
muscle_grouping = muscle_grouping.drop('tissue', axis=1)
print(muscle_grouping.shape)
muscle_grouping.head()

NameError: name 'unified_annotations' is not defined

## IGNORE FOR NOW: Concatenate muscle and blood since both use `cell_ontology_type`

In [ ]:
# blood_muscle = pd.concat([blood_grouping, muscle_grouping])
# blood_muscle = blood_muscle.loc[~blood_muscle.index.duplicated()]
# blood_muscle.head()

### Merge new renamed, minimal obs with blood muscle for `narrow_group`, `broad_group`

In [ ]:
minimal_obs_with_groupings = minimal_obs.merge(
    muscle_grouping, how="left", left_on="free_annotation", right_index=True
)
# minimal_obs_with_groupings.index = minimal_obs_with_groupings.index.astype(str)
print(minimal_obs_with_groupings.shape)
minimal_obs_with_groupings.head()

In [ ]:
minimal_obs_with_groupings.groupby(
    ["tissue", "compartment_group", "broad_group", "narrow_group"], observed=True
).size().to_frame()

# Add new metadata

In [ ]:
# new_var.index = new_var.index.str.decode('utf-8')
# new_var['ensembl_id'] = new_var['ensembl_id'].str.decode('utf-8')
new_var.head()

In [ ]:
sapiens_10x_no_diaphragm_pilot2.var = new_var
sapiens_10x_no_diaphragm_pilot2.var.head()

In [ ]:
sapiens_10x_no_diaphragm_pilot2.var.sample(10)

In [ ]:
sapiens_10x_no_diaphragm_pilot2.obs = minimal_obs_with_groupings

sapiens_10x_no_diaphragm_pilot2

In [ ]:
sapiens_10x_no_diaphragm_pilot2.var.head()

In [ ]:
sapiens_10x_no_diaphragm_pilot2.var.dtypes

In [ ]:
sapiens_10x_no_diaphragm_pilot2.obs.dtypes

In [ ]:
sapiens_10x_no_diaphragm_pilot2.obs.head()

## Write h5ad with new metadata to file

In [ ]:
%%time
sapiens_10x_no_diaphragm_pilot2.write(
    os.path.join(
        "/home/olga/data_lg/data_sm_copy/tabula-sapiens/data-objects/",
        "tabula-sapiens_P1P2_10X_SS2_Muscle_updated-annotations__minimal_no_pilot2_diaphragm__muscle_update_august2021.h5ad",
    )
)

## Check cell type presence

In [ ]:
def check_celltype_presence(adata, celltypes=('fast muscle cell',), celltype_col='narrow_group'):
    for celltype in celltypes:
        try:
            assert celltype in set(adata.obs[celltype_col])
        except:
            raise AssertionError(f"Celltype '{celltype}' not found in column '{celltype_col}'")

check_celltype_presence(sapiens_10x_no_diaphragm_pilot2)

In [ ]:
sorted(sapiens_10x_no_diaphragm_pilot2.obs.tissue.unique())

In [ ]:
sapiens_10x_no_diaphragm_pilot2.obs.query("tissue == 'Muscle'").narrow_group.value_counts().sort_index()

# Subset to 1:1 orthologs

## Read orthologs

In [ ]:
csv = os.path.join(
    "/home/olga/googledrive/TabulaMicrocebus/data/orthologous-genes",
    "ncbi_mgi_ensembl__mouse-lemur_human_mouse__orthologs__gene_names__one2one.csv",
)

gene_orthologies = pd.read_csv(csv, index_col=0)
print(gene_orthologies.shape)
gene_orthologies.head()

## Use only genes in the 1:1:1 ortholog dataframes

In [ ]:
gene_mask = sapiens.var.index.isin(gene_orthologies['human__gene_name'].values)
print(gene_mask.sum())

# Need to copy so it's not an ArrayView object
sapiens_10x_no_diaphragm_pilot2_one2one = sapiens_10x_no_diaphragm_pilot2[:, gene_mask].copy()
sapiens_10x_no_diaphragm_pilot2_one2one

In [ ]:
sapiens_10x_no_diaphragm_pilot2_one2one.var = gene_orthologies.set_index('human__gene_name').loc[sapiens_10x_no_diaphragm_pilot2_one2one.var.index]
sapiens_10x_no_diaphragm_pilot2_one2one.var.head()

## Write sapiens subset to file

In [ ]:
%%time

one2one_h5ad = os.path.join(
    outdir_local, "tabula_sapiens__10x__human__one2one_orthologs__august2021.h5ad"
)
sapiens_10x_no_diaphragm_pilot2_one2one.write(one2one_h5ad)

In [ ]:
! ls -lha $one2one_h5ad